<a href="https://colab.research.google.com/github/ShenggKai/kaggle_TempoRun/blob/master/Notebook/ShengKai_TempoRun_InformationRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://i.imgur.com/MZQ1o3n.png" alt="winer"/>
</center>

[[AI TEMPO RUN] Retrieval of Song Lyrics](https://www.kaggle.com/c/temporun-retrieval-of-song-lyrics/)

# My team

* ✈ Name: Meowmeow  
* 🕶 Team members: Nguyễn Quốc Đạt - [ShengKaii](https://www.kaggle.com/shengkaii)

# Setup

## Install the Kaggle library

In [ ]:
!pip install kaggle

Download API Credentials 

In [25]:
!wget -O kaggle.json https://raw.githubusercontent.com/ShenggKai/kaggle_TempoRun/master/API_token/kaggle.json

--2022-03-15 09:12:26--  https://raw.githubusercontent.com/ShenggKai/kaggle_TempoRun/master/API_token/kaggle.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65 [text/plain]
Saving to: ‘kaggle.json’

kaggle.json         100%[===================>]      65  --.-KB/s    in 0s      

2022-03-15 09:12:26 (2.14 MB/s) - ‘kaggle.json’ saved [65/65]



## Install and import required library

In [35]:
!pip install rank-bm25

In [36]:
!pip install -q tf-models-official==2.3.0

     |████████████████████████████████| 840 kB 4.0 MB/s 
     |████████████████████████████████| 1.2 MB 50.7 MB/s 
     |████████████████████████████████| 234 kB 42.9 MB/s 
     |████████████████████████████████| 352 kB 46.3 MB/s 
     |████████████████████████████████| 47.8 MB 1.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.7 MB/s 
     |████████████████████████████████| 99 kB 7.6 MB/s 
     |████████████████████████████████| 462 kB 56.8 MB/s 


In [38]:
import os
import pprint as pp
import numpy as np
import pandas as pd
import json
import tensorflow as tf 
from gensim.corpora import WikiCorpus
from rank_bm25 import BM25Okapi, BM25Plus

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

from tqdm import tqdm

## Kiểm tra thư mục

[How to Load Kaggle Datasets Directly into Google Colab?](https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/)

In [27]:
print("Hien tai ta dang o thu muc:")
!pwd

Hien tai ta dang o thu muc:
/content


In [26]:
ls

kaggle.json  sample_data/


Make a directory named “.kaggle”

In [28]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Copy the “kaggle.json” into this new directory



In [29]:
! cp kaggle.json ~/.kaggle/

Allocate the required permission for this file.

In [30]:
! chmod 600 ~/.kaggle/kaggle.json

# Download dataset

In [33]:
# Kiem tra version
!kaggle --version

Kaggle API 1.5.12


In [32]:
!kaggle competitions download -c temporun-retrieval-of-song-lyrics

  0% 0.00/1.35M [00:00<?, ?B/s]
100% 1.35M/1.35M [00:00<00:00, 111MB/s]


Bởi vì file tải về có dạng zip nên phải giải nén file

In [34]:
!unzip /content/temporun-retrieval-of-song-lyrics.zip

Archive:  /content/temporun-retrieval-of-song-lyrics.zip
  inflating: 2335_Songs.json         
  inflating: 500_queries.json        
  inflating: sample_submission.csv   


# Preliminaries & Corpus Creation

In [43]:
df = pd.read_json (r'/content/2335_Songs.json')
df.to_csv (r'/content/sample_data/data.csv', index = None)

In [44]:
df["song_id"].head(100)

0       0
1       1
2       2
3       3
4       4
     ... 
95    155
96    156
97    157
98    158
99    159
Name: song_id, Length: 100, dtype: int64

In [39]:
CORPUS_SONG_JSON = "2335_Songs.json"
QUERIES_JSON = "500_queries.json"

In [41]:
with open(CORPUS_SONG_JSON, 'r', encoding='utf-8') as fo:
  corpus_data = json.load(fo)

with open(QUERIES_JSON, 'r', encoding='utf-8') as fo:
  queries_test = json.load(fo)

In [42]:
type(corpus_data)

list

In [45]:
print(corpus_data[76])

{'author': 'Phúc Bồ', 'title': 'Ngắm Trăng', 'lyrics': 'Chỉ có anh với những nốt nhạc vang lên trong màn đen đêm buồn\nThời gian làm ta lặng im tựa như cây lá bên đường\nCô đơn phải chăng là mẫu đàn ông em muốn\nHình như là mặt trăng cũng có đôi tai\nNgồi im và nghe anh hát\nSống trong giai điệu như là Maroon 5\nVang lên những ngày xanh mát\nVà anh đang ngồi đây như 1 nhà thơ nào đó cứ đòi bán ánh trăng đi\nvà đã lỡ yêu quá nhiều,\ntâm hồn như là 1 cõi để cho gió cuốn phăng đi,\ntâm trạng rối bời như lời nhạc của Yanbi\nNow im singing with the moonlight\nChỉ là bữa tiệc cùng cô đơn vào thường ngày\nIm drinking with the late night\nĐể được nói hết không cần biết đúng hay sai\nMột giấc mơ đẹp có thể nằm trên sofa\nMột cơn ác mộng khi em hỏi về cô ta\nMột chuyện tình đẹp không mang đến sự vĩnh cửu\nNhưng ai cũng nhớ về tiếng đàn của Lorca\nLời ca anh đang hòa vào không trung cùng những ánh sao trên trời.\nMang giấc mơ anh trở về nơi mà hai ta luôn chờ đợi\nĐưa ánh trăng đi tìm từng hơi th